In [1]:
from datasets import Dataset

In [2]:
val_dataset = Dataset.load_from_disk("../oqa_v1.0_shuffled_split/bin/val")
#val_dataset = Dataset.load_from_disk("tmp/bin/val")

In [3]:
val_dataset

Dataset({
    features: ['question', 'context', 'answers', 'answer_sentence', 'topic', 'reference', 'id'],
    num_rows: 55
})

In [4]:
%load_ext autoreload
%autoreload 2

In [24]:
from models.llm_inference import GpuInference
tt5 = GpuInference(model_checkpoint = "google/t5-efficient-tiny",
             tokenizer_checkpoint = "google/t5-efficient-tiny",
             int8=False,
             dataset=val_dataset,
            prompt_fmt="uniqa")
prompts = tt5.get_prompts()

from accelerate import Accelerator

In [33]:
prompts[0]

{'answer': 'an elastic should be stretched to 3 times its passive inner diameter, and the static force of resistance should then be recorded in grams or ounces from a mechanical strain gauge or similar instrument',
 'prompt': "regarding orthodontic elastics, what is the rule of 3 used by manufacturers to measure elastic force?\nwhile most manufacturers assign a biomechanical force value, usually in ounces, to each of their elastic categories, the values themselves show an abundance of built-in elasticity. for example, in michael langlade's specialized book optimization of orthodontic elastics, the reader is provided a framework for associating the marketing categories of light, medium, and heavy with specific reference values. a medium force, says langlade, falls within the range of 6 to 10 ounces (170 to 283 grams). this is not only an overly broad range, but it is also significantly above the force values for medium stated by most manufacturers. this incongruence is a salient example

In [34]:
a = tt5.genseq(prompts)

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

  0%|          | 0/55 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
a[0].size()

torch.Size([5, 4])

In [39]:
ans = [tt5.tokenizer.decode(x, skip_special_tokens=True) for x in a[3]]
ans

['',
 'incisor display at rest and smile were evaluated separately. this study analyzed the difference between men and women. ackerman et al. with increasing age, we reported that this effect was significantly reduced in women and men.: i.e., vs.',
 'incisor display at rest and smile were evaluated separately. this study analyzed the difference between men and women. ackerman et al. with increasing age, we found that the effect of age increased significantly.: i.e., h..:',
 'incisor display at rest and smile were evaluated separately. this study analyzed the difference between men and women. ackerman et al. with increasing age, we reported that this effect was significantly reduced in women and men.: i.e., p.m',
 'incisor display at rest and smile were evaluated separately. this study analyzed the difference between men and women. ackerman et al. with increasing age, we reported that this effect was significantly reduced in women and men.: i.e., hyster']

In [29]:
ans

['', '', 'x', 'to', '']